In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'; drop "Application_Type" and "Special_Considerations".
application_df = application_df.drop(["EIN", "NAME", "APPLICATION_TYPE", "SPECIAL_CONSIDERATIONS"], axis=1)
application_df.head()

,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,Independent,C1000,ProductDev,Association,1,0,5000,1
1,Independent,C2000,Preservation,Co-operative,1,1-9999,108590,1
2,CompanySponsored,C3000,ProductDev,Association,1,0,5000,0
3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,6692,1
4,Independent,C1000,Heathcare,Trust,1,100000-499999,142590,1


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

AFFILIATION          6
CLASSIFICATION      71
USE_CASE             5
ORGANIZATION         4
STATUS               2
INCOME_AMT           9
ASK_AMT           8747
IS_SUCCESSFUL        2
dtype: int64

In [4]:
# Look at CLASSIFICATION value counts for binning
class_counts = application_df.CLASSIFICATION.value_counts()
class_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [5]:
# Determine which values to replace if counts are less than 1250?
replace_class = list(class_counts[class_counts < 1250].index)


# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [6]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()

In [7]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

C:\Users\Owner\anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,CLASSIFICATION_C1000,CLASSIFICATION_C1200,CLASSIFICATION_C2000,CLASSIFICATION_C2100,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [8]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df,left_index=True, right_index=True)
application_df = application_df.drop(application_cat,1)
application_df.head()

C:\Users\Owner\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,STATUS,ASK_AMT,IS_SUCCESSFUL,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,CLASSIFICATION_C1000,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1,5000,1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,108590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,5000,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,6692,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,142590,1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [9]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

C:\Users\Owner\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [11]:
# Attempt #1 - Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer. 
number_input_features1 = len(X_train[0])
hidden_nodes_layer11 = 80
hidden_nodes_layer21 = 40
hidden_nodes_layer31 = 30


nn1 = tf.keras.models.Sequential()

# First hidden layer
nn1.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer11, input_dim=number_input_features1, activation="relu")
)

# Second hidden layer
nn1.add(tf.keras.layers.Dense(units=hidden_nodes_layer21, activation="relu"))

# Third hidden layer
nn1.add(tf.keras.layers.Dense(units=hidden_nodes_layer31, activation="leaky_relu"))


# Output layer
nn1.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                2640      
                                                                 
 dense_1 (Dense)             (None, 40)                3240      
                                                                 
 dense_2 (Dense)             (None, 30)                1230      
                                                                 
 dense_3 (Dense)             (None, 1)                 31        
                                                                 
Total params: 7,141
Trainable params: 7,141
Non-trainable params: 0
_________________________________________________________________


In [12]:
# Attempt # 1 Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints1/",exist_ok=True)
checkpoint_path = "checkpoints1/weights.{epoch:02d}.hdf5"

In [13]:
# Attempt #1 Compile the model
nn1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [14]:
# Attempt # 1 Create a callback that saves the model's weights every 5 epochs
cp_callback1 = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5*804)

In [15]:
# Attempt # 1 Train the model
fit_model = nn1.fit(X_train_scaled,y_train,epochs=50,callbacks=[cp_callback1])

Epoch 1/50
804/804 [==============================] - 3s 3ms/step - loss: 0.5941 - accuracy: 0.6996
Epoch 2/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5805 - accuracy: 0.7062
Epoch 3/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5783 - accuracy: 0.7096
Epoch 4/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5774 - accuracy: 0.7090
Epoch 5/50
789/804 [============================>.] - ETA: 0s - loss: 0.5768 - accuracy: 0.7102
Epoch 00005: saving model to checkpoints1\weights.05.hdf5
804/804 [==============================] - 2s 3ms/step - loss: 0.5761 - accuracy: 0.7107
Epoch 6/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5758 - accuracy: 0.7121
Epoch 7/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5759 - accuracy: 0.7100
Epoch 8/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5748 - accuracy: 0.7096
Epoch 9/50
804/804 [==============================] - 2s 

In [16]:
# Attempt # 1 Evaluate the model using the test data
model_loss, model_accuracy = nn1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5810 - accuracy: 0.7066 - 836ms/epoch - 3ms/step
Loss: 0.581016480922699, Accuracy: 0.7065889239311218


In [17]:
# Attempt # 1 Export our model to HDF5 file
nn1.save("AlphabetSoupCharity_Optimization1.h5")

In [18]:
# Attempt #2 - Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer. 
number_input_features2 = len(X_train[0])
hidden_nodes_layer12 = 35
hidden_nodes_layer22 = 15


nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer12, input_dim=number_input_features2, activation="relu")
)

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer22, activation="leaky_relu"))


# Output layer
nn2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 35)                1155      
                                                                 
 dense_5 (Dense)             (None, 15)                540       
                                                                 
 dense_6 (Dense)             (None, 1)                 16        
                                                                 
Total params: 1,711
Trainable params: 1,711
Non-trainable params: 0
_________________________________________________________________


In [19]:
# Attempt # 2 Define the checkpoint path and filenames
os.makedirs("checkpoints2/",exist_ok=True)
checkpoint_path = "checkpoints2/weights.{epoch:02d}.hdf5"

In [20]:
# Attempt #2 Compile the model
nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [21]:
# Attempt # 2 Create a callback that saves the model's weights every 5 epochs
cp_callback2 = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5*804)

In [22]:
# Attempt # 2 Train the model
fit_model = nn2.fit(X_train_scaled,y_train,epochs=50,callbacks=[cp_callback2])

Epoch 1/50
804/804 [==============================] - 3s 3ms/step - loss: 0.5989 - accuracy: 0.6966
Epoch 2/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5823 - accuracy: 0.7049
Epoch 3/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5792 - accuracy: 0.7075
Epoch 4/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5774 - accuracy: 0.7077
Epoch 5/50
802/804 [============================>.] - ETA: 0s - loss: 0.5769 - accuracy: 0.7096
Epoch 00005: saving model to checkpoints2\weights.05.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5768 - accuracy: 0.7096
Epoch 6/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5758 - accuracy: 0.7118
Epoch 7/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5755 - accuracy: 0.7111: 0s - loss: 0.5737 - ac
Epoch 8/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5747 - accuracy: 0.7107
Epoch 9/50
804/804 [=============

In [23]:
# Attempt # 2 Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5798 - accuracy: 0.7036 - 544ms/epoch - 2ms/step
Loss: 0.5798234343528748, Accuracy: 0.7035568356513977


In [24]:
# Attempt # 2 Export our model to HDF5 file
nn2.save("AlphabetSoupCharity_Optimization2.h5")

In [25]:
# Attempt #3 - Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer. 
number_input_features3 = len(X_train[0])
hidden_nodes_layer13 = 35
hidden_nodes_layer23 = 30
hidden_nodes_layer33 = 20
hidden_nodes_layer43 = 15


nn3 = tf.keras.models.Sequential()

# First hidden layer
nn3.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer13, input_dim=number_input_features3, activation="relu")
)

# Second hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer23, activation="leaky_relu"))

# Third hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer33, activation="relu"))

# Fourth hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer43, activation="relu"))


# Output layer
nn3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn3.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 35)                1155      
                                                                 
 dense_8 (Dense)             (None, 30)                1080      
                                                                 
 dense_9 (Dense)             (None, 20)                620       
                                                                 
 dense_10 (Dense)            (None, 15)                315       
                                                                 
 dense_11 (Dense)            (None, 1)                 16        
                                                                 
Total params: 3,186
Trainable params: 3,186
Non-trainable params: 0
_________________________________________________________________


In [26]:
# Attempt # 3 Define the checkpoint path and filenames
os.makedirs("checkpoints3/",exist_ok=True)
checkpoint_path = "checkpoints3/weights.{epoch:02d}.hdf5"

In [27]:
# Attempt #3 Compile the model
nn3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [28]:
# Attempt # 3 Create a callback that saves the model's weights every 5 epochs
cp_callback3 = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5*804)

In [29]:
# Attempt # 3 Train the model
fit_model = nn3.fit(X_train_scaled,y_train,epochs=50,callbacks=[cp_callback3])

Epoch 1/50
804/804 [==============================] - 3s 3ms/step - loss: 0.6008 - accuracy: 0.6910
Epoch 2/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5802 - accuracy: 0.7080
Epoch 3/50
804/804 [==============================] - 3s 3ms/step - loss: 0.5777 - accuracy: 0.7109
Epoch 4/50
804/804 [==============================] - 3s 3ms/step - loss: 0.5770 - accuracy: 0.7111
Epoch 5/50
801/804 [============================>.] - ETA: 0s - loss: 0.5764 - accuracy: 0.7095
Epoch 00005: saving model to checkpoints3\weights.05.hdf5
804/804 [==============================] - 2s 3ms/step - loss: 0.5763 - accuracy: 0.7093
Epoch 6/50
804/804 [==============================] - 3s 4ms/step - loss: 0.5753 - accuracy: 0.7102
Epoch 7/50
804/804 [==============================] - 3s 4ms/step - loss: 0.5747 - accuracy: 0.7123
Epoch 8/50
804/804 [==============================] - 3s 4ms/step - loss: 0.5741 - accuracy: 0.7141
Epoch 9/50
804/804 [==============================] - 2s 

In [30]:
# Attempt # 3 Evaluate the model using the test data
model_loss, model_accuracy = nn3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5787 - accuracy: 0.7054 - 638ms/epoch - 2ms/step
Loss: 0.5786973237991333, Accuracy: 0.7054227590560913


In [31]:
# Attempt # 3 Export our model to HDF5 file
nn3.save("AlphabetSoupCharity_Optimization3.h5")